# Fetch Assessment

In [1]:
from utils import *
from dataset import *

In [2]:
dataset = TaskDataset(data=None)

Using cache found in C:\Users\yashv/.cache\torch\hub\huggingface_pytorch-transformers_main


# Task 1
Sentence Embedding Model, architecture:
 - Embedding Layer 
 - Positional Encoding
 - Transformer Encoder Layer
 - Mean Pooling
 - Linear Layer (generates embedding)

In [3]:
# initialize models
n_layers = 1 # The number of transformer encoder layers
d_model = 512 # The input embedding dimension
embed_size = 300 # The output embedding size
nhead = 8 # The number of attention heads in encoder layer
task_type = "se" # The type of task (sentence embedding or sentiment classification)
vocab_size = dataset.vocab_size # The size of the vocabulary (using hugging face bert tokenizer)
base_model, se_task_model = init_model(vocab_size, n_layers, d_model, nhead, embed_size, task_type)

In [4]:
# Generate sample input
input_data = torch.randint(0, dataset.vocab_size, (2, 100))

# Forward pass
base_output = base_model(input_data)
task_output = se_task_model(base_output)

print("Base model output:", base_output.shape)
print("Task model output:", task_output.shape)

# Generate input based on text
text1 = "How, are you?"
text2 = "I am good"

input_data1 = dataset.encode(text1)
input_data2 = dataset.encode(text2)

# Forward pass
base_output1 = base_model(input_data1)
base_output2 = base_model(input_data2)
task_output1 = se_task_model(base_output1)
task_output2 = se_task_model(base_output2)

print("Base model output 1:", base_output1.shape)
print("Base model output 2:", base_output2.shape)
print("Task model output 1:", task_output1.shape)   
print("Task model output 2:", task_output2.shape)
print("Task model output 2:", task_output2)

Base model output: torch.Size([2, 100, 512])
Task model output: torch.Size([2, 300])
Base model output 1: torch.Size([1, 7, 512])
Base model output 2: torch.Size([1, 5, 512])
Task model output 1: torch.Size([1, 300])
Task model output 2: torch.Size([1, 300])
Task model output 2: tensor([[-0.0981,  0.0113,  0.0783, -0.0984, -0.1121,  0.0839, -0.0039, -0.0043,
         -0.0016, -0.0034, -0.0087, -0.0402,  0.1083, -0.0036,  0.0044,  0.0531,
         -0.0384, -0.0287, -0.0513, -0.0299,  0.0021, -0.1606,  0.0202,  0.0958,
         -0.0749, -0.0230, -0.0574,  0.0605,  0.0416, -0.0383,  0.0436, -0.1180,
          0.0140, -0.0155, -0.0188,  0.0909,  0.0620, -0.0758,  0.1260,  0.0193,
         -0.0019,  0.1674,  0.0666, -0.1502, -0.0950, -0.0204,  0.0168,  0.0302,
         -0.0038,  0.0521,  0.0924,  0.0821,  0.0929,  0.0542, -0.0457,  0.0146,
         -0.0209, -0.0112,  0.0023, -0.0243, -0.0911,  0.0149, -0.0301, -0.0309,
          0.0569, -0.0143, -0.0088, -0.0308, -0.0692,  0.0741, -0.0068, 

# Task 2
Sentiment Classification Model, architecture:
 - Embedding Layer
 - Positional Encoding
 - Transformer Encoder Layer
 - Mean Pooling
 - Linear Layer (3 classes, Positive, Negative, Neutral) ( can be trained using cross entropy loss)

In [5]:
task_type = "sc"
base_model, sc_task_model = init_model(vocab_size, n_layers, d_model, nhead, embed_size, task_type)

In [6]:
# Generate sample input
input_data = torch.randint(0, dataset.vocab_size, (2, 100))

# Forward pass
base_output = base_model(input_data)
task_output = sc_task_model(base_output)
task_output = nn.Softmax(dim=1)(task_output).detach().numpy()

print("Base model output:", base_output.shape)
print("Task model output:", task_output.shape)
print("Task model output:", task_output)

# Generate input based on text
text1 = "How, are you?"
text2 = "I am good"

input_data1 = dataset.encode(text1)
input_data2 = dataset.encode(text2)

# Forward pass
base_output1 = base_model(input_data1)
base_output2 = base_model(input_data2)
task_output1 = sc_task_model(base_output1)
task_output2 = sc_task_model(base_output2)

task_output1 = nn.Softmax(dim=1)(task_output1).detach().numpy()
task_output2 = nn.Softmax(dim=1)(task_output2).detach().numpy()

print("Base model output 1:", base_output1.shape)
print("Base model output 2:", base_output2.shape)
print("Task model output 1:", task_output1.shape)   
print("Task model output 2:", task_output2.shape)

Base model output: torch.Size([2, 100, 512])
Task model output: (2, 3)
Task model output: [[0.245916   0.42255494 0.33152908]
 [0.21411856 0.4375086  0.3483728 ]]
Base model output 1: torch.Size([1, 7, 512])
Base model output 2: torch.Size([1, 5, 512])
Task model output 1: (1, 3)
Task model output 2: (1, 3)


# Task 4

Assuming we are training the sentence embedding model, and the sentiment classification model, simulatenously. Then each model can have different learning rates, as well for embedding in transformer will add different learning rate.

* Embedding Layer: Set a learning rate of 1e-3.
  
  -  This is a low-level layer responsible for capturing foundational features, so a finer learning rate helps prevent drastic changes that could drift away from good initial representations.
* Transformer Layer: Use a learning rate of 5e-3.
   
   - As a middle layer, this part of the model focuses on generalizing patterns from the dataset. A slightly higher learning rate allows it to adapt faster, helping it develop better generalizations over time.
* Sentence Embedding and Sentiment Classification Layers: Set a learning rate of 1e-2.
  - These layers capture task-specific, high-level features crucial for task understanding. A higher learning rate here allows faster learning of these details, which is important to avoid long convergence times.

In [7]:
print(base_model.named_parameters)

<bound method Module.named_parameters of BaseModel(
  (embed): Embedding(30522, 512)
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
    )
    (linear1): Linear(in_features=512, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=512, bias=True)
    (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)


In [8]:
import torch.optim as optim

In [9]:
parameters = []
base_model_transformer_lr = 5e-3
base_model_embedding_lr = 1e-3

for name, param in base_model.named_parameters():
    if "embed" in name:
        parameters.append({"params": param, "lr": base_model_embedding_lr})
    else:
        parameters.append({"params": param, "lr": base_model_transformer_lr})

se_lr = 1e-2
sc_lr = 1e-2

parameters.append(
    {"params" : se.parameters(), "lr" : se_lr}
)

parameters.append(
    {"params" : sc.parameters(), "lr" : sc_lr}
)

optimizer = optim.Adam(parameters)

NameError: name 'se' is not defined